In [7]:
import pandas as pd
import pickle
import numpy as np
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import re
import time
from geopy.extra.rate_limiter import RateLimiter

In [8]:
pickle_in = open("parrot.pkl","rb")
links = pickle.load(pickle_in)

In [9]:
name = []
address = []
features = {'Level Access and Automatic Doors (or No Doors)':[], 'Ramped/Sloped Access, and/or Manual Doors':[],
           'Ramped/Sloped Access that is Long / Steep (may have Manual Doors)':[], 'Mobility Impaired Walker':[],
           'On-Site Parking':[], 'Off-Site Parking':[], 'Blue Badge/Accessible Parking Bays':[], 'Designated Drop-Off Point':[],
           'Accessible Toilet':[], 'Ambulant Toilet':[], 'Step Free Standard Toilet':[], 'Gender Neutral Toilet':[],
           'Baby Changing Facility':[], 'Changing Place':[], 'Accessible Changing Room':[], 'Accessible Fitting Room':[],
           'Accessible Accommodation':[], 'Hoist':[], 'Accessible Shower':[], 'Seat(s) Available':[], 'Safe or Quiet Space':[],
           'Sign Language':[], 'Assistive Listening':[], 'Audio Description':[], 'Audio Format Information':[], 'Large Print':[],
           'Braille':[], 'Facilities for Assistance Dogs':[], 'Disability Awareness/Equality Training':[], 'Dementia Friendly':[],
           'Companion/Carer Discount':[], 'Home Visit':[]}

In [10]:
url = links[0]
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tag = type(soup.find('h1', 'u-h2 heading u-colour-primary'))

In [ ]:
# find venue details

for i in range(len(links)):
    url = links[i]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if type(soup.find('h1', 'u-h2 heading u-colour-primary')) == tag:
        name.append(soup.find('h1', 'u-h2 heading u-colour-primary').get_text())
        address.append(soup.find('p').get_text())
    
        page2 = requests.get(url+'/access_symbols/summary')
        soup2 = BeautifulSoup(page2.content, 'html.parser')
    
        temp = []
        for j in range(len(soup2.find_all('span'))):
            temp.append(soup2.find_all('span')[j].get_text().strip())
            
        for k in features.keys():
            if k in temp:
                features[k].append(1)
            else:
                features[k].append(0)
    print(i, 'complete')

0 complete
1 complete
2 complete
3 complete
4 complete
5 complete
6 complete
7 complete
8 complete
9 complete
10 complete
11 complete
12 complete
13 complete
14 complete
15 complete
16 complete
17 complete
18 complete
19 complete
20 complete
21 complete
22 complete
23 complete
24 complete
25 complete
26 complete
27 complete
28 complete
29 complete
30 complete
31 complete
32 complete
33 complete
34 complete
35 complete
36 complete
37 complete
38 complete
39 complete
40 complete
41 complete
42 complete
43 complete
44 complete
45 complete
46 complete
47 complete
48 complete
49 complete
50 complete
51 complete
52 complete
53 complete
54 complete
55 complete
56 complete
57 complete
58 complete
59 complete
60 complete
61 complete
62 complete
63 complete
64 complete
65 complete
66 complete
67 complete
68 complete
69 complete
70 complete
71 complete
72 complete
73 complete
74 complete
75 complete
76 complete
77 complete
78 complete
79 complete
80 complete
81 complete
82 complete
83 complete
84

In [25]:
df = pd.DataFrame({'Name': name, 'Address': address, 'Full': [m+', '+str(n) for m,n in zip(name,address)]})
df = df.join(pd.DataFrame.from_dict(features))

In [26]:
#pickle_out = open("df.pickle","wb")
#pickle.dump(df, pickle_out)
#pickle_out.close()

#pickle_out = open("name.pickle","wb")
#pickle.dump(name, pickle_out)
#pickle_out.close()

#pickle_out = open("address.pickle","wb")
#pickle.dump(address, pickle_out)
#pickle_out.close()

#pickle_out = open("features.pickle","wb")
#pickle.dump(features, pickle_out)
#pickle_out.close()

In [27]:
pickle_in = open("df.pickle","rb")
df = pickle.load(pickle_in)

In [37]:
latitude = []
longitude = []

'0.31 IBERS, IBERS New Build, Aberystwyth University, Penglais Campus, Aberystwyth, SY23 3DA'

In [204]:
geolocator = Nominatim(timeout=5)

for i in range(11790, len(df)):
    location = geolocator.geocode(df['Full'][i])
    if location is None:
        a = re.split('[?.,\-!+()]', df['Full'][i])
        location2 = geolocator.geocode(a[0]+', '+a[-1])
        if location2 is None:
            latitude.append('NA')
            longitude.append('NA')
        else:
            latitude.append(location2.latitude)
            longitude.append(location2.longitude)      
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    print(i, 'complete')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [205]:
i

11790

In [206]:
pickle_out = open("longitude.pickle","wb")
pickle.dump(longitude, pickle_out)
pickle_out.close()

pickle_out = open("latitude.pickle","wb")
pickle.dump(latitude, pickle_out)
pickle_out.close()